# Visualizar terremotos

Estos datos sobre terremotos son cortesía de [USGS](https://earthquake.usgs.gov).

{query_address}

Escribe una dirección para ordenar los terremotos recientes por distancia.

In [ ]:
target_folder = '/tmp'
query_address = 'David, Chiriquí'

In [ ]:
from geopy.geocoders import GoogleV3
g = GoogleV3()
query_location = g.geocode(query_address)
query_point = query_location.point
query_point

In [ ]:
import pandas as pd

In [ ]:
table_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.csv'
table = pd.read_csv(table_url, parse_dates=['time'])
table.ix[0]

In [ ]:
table = table[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
table.ix[0]

In [ ]:
from geopy.distance import vincenty as get_distance

def get_distance_in_km(x):
    return pd.Series(get_distance(query_point, (x.latitude, x.longitude)).kilometers)
    
table['distanceInKm'] = table.apply(get_distance_in_km, axis=1)
table = table.sort_values(['distanceInKm'])
table[:3]

In [ ]:
from datetime import datetime
now = datetime.utcnow()

def get_elapsed_time_in_negative_seconds(x):
    return -1 * (now - x).seconds

geotable = table.copy()
geotable['fillReds'] = geotable['time'].apply(get_elapsed_time_in_negative_seconds)
geotable['radiusInPixelsRange3-27'] = geotable['mag']
geotable.ix[0]

In [ ]:
from os.path import join

table_path = join(target_folder, 'terremotos.csv')
table.to_csv(table_path, index=False)
print('terremotos_table_path = %s' % table_path)

geotable_path = join(target_folder, 'terremotos-mapa.csv')
geotable.to_csv(geotable_path, index=False)
print('terremotos_satellite_geotable_path = %s' % geotable_path)

# Terremotos visualizados

{terremotos_table}

{terremotos_satellite_geotable}

El radio del círculo indica la magnitud.

El color del círculo indica el tiempo.

[USGS](https://earthquake.usgs.gov) ofrece más tablas sobre terremotos recientes.